In [5]:
import pandas as pd
from datetime import datetime
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import win32com.client as win32
pd.set_option('mode.chained_assignment', None)
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

# Function to send emails
def send_email(to_email, cc_email, subject, message):
    # Your email configuration for Outlook
    outlook = win32.Dispatch('Outlook.Application')
    mail = outlook.CreateItem(0)
    mail.Subject = subject
    mail.HTMLBody = message
    mail.To = to_email
    mail.CC = cc_email
    
    # Send the email
    mail.Send()

# Get the current date dynamically
current_date = datetime.now().strftime("%d-%m-%Y")
csv_file_path = f'Mis_report_{current_date}.csv'

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file_path)

# List of columns to be removed
columns_to_remove = ['Sub Total','eWay no','SrNo', 'Mode', 'NOP', 'AW', 'CW', 'Consignor Invoice No', 'Consignor Invoice Value', 'Bill Type',
                     'TAT', 'EDD Date', 'RTO Date', 'RTO Reason', 'Franchise Code', 'Franchise Name',
                     'Master Franchise Name', 'Eway Expiry date', 'Pickupinscan date & time',
                     'pickupinscan branch', 'Booking Branch Out scan Date & Time',
                     'Delivery branch In-scan Date & Time', 'DRS Date & time', 'Regular/ODA', 'PRQ No',
                     'Pickup Genarte Date', 'Pickup Requested Date', 'PRQ Closed Date', 'PRQ Comment',
                     'Cash Invoice No', 'Invoice Date', 'PayBy', 'Payment Ref No', 'Coloader Name', 'CD No',
                     'CD Outscan', 'CD Inscan', 'Product Desc', 'Type Of Package', 'POD Uploaded Date & Time','Sales Person Name','Sales Person Branch','Freight','Handling','Pickup','ODA','Insurance','COD','AWB.1','Other','Topay','Appoint','FOV','Total','Fuel','Subtotal']

# Drop the specified columns
df = df.drop(columns=columns_to_remove, axis=1)

values_to_remove = [500128742, 100024181, 500128371, 100035785, 500124922, 100018239, 100023775,
    500124023, 500124031, 500124036, 500124038, 100023873, 100023732, 100023736,
    100023971, 100023974, 100025722, 100025873, 100008655, 'FBI122336', 100008544,
    100008729, 100008731, 'FBI121307', 100016964, 100012560, 100014024, 100014033,
    100015138, 50118704, 200001377, 100018521, 100018195, 100015103, 200001369
]
# Remove the specified values from the DataFrame
df = df[~df['AWB'].astype(str).isin(map(str, values_to_remove))]

# Filter rows where 'Current Status' is 'Delivered'
df_delivered = df[df['Current Status'] == 'Delivered']

# Remove rows where 'POD Status' is 'Yes'
df_filtered = df_delivered[df_delivered['POD Status'] != 'Yes']

# Define city(Mostly Branch)-specific data processing functions
def process_city(city_name, branches):
    city_filtered = df_filtered[df_filtered['DRS Branch'].isin(branches)]

    # Convert the 'Booking Date' column to datetime
    city_filtered['Booking Date'] = pd.to_datetime(city_filtered['Booking Date'], dayfirst=True)

    # Filter rows for July, August, and September
    df_july_august_sep = city_filtered[
        (city_filtered['Booking Date'].dt.month.isin([9,10,11,12])) &
        (city_filtered['Booking Date'].dt.year == 2023)
    ]

    # Save the filtered DataFrame to a new CSV file
    # output_csv_path = f'Excel Data/{city_name}_SOFTCOPY_POD_PENDING_UPDATION.csv'
    output_csv_path = f'Excel Data/{city_name}_SOFTCOPY_POD_PENDING_UPDATION.csv'
    city_filtered.to_csv(output_csv_path, index=False)

    return df_july_august_sep

# Define city names and branch lists
cities = {
    'Hyderabad': ['Karim Nagar VHYD','Hyderabad_BegamPet ','Hyderabad_BegamPet','Hyderabad_Hyderabad','Hyderabad_Siripur-Khagaj Nagar','Hyderabad_Kothakota','Hyderabad_Huzur Nagar','Hyderabad_Kadiri','Hyderabad_Hindupur','Hyderabad_Kakinada','Hyderabad_nalgonda','Hyderabad_CUDDAPAH BRANCH','Hyderabad_Tadipatri MAIN BRANCH','Hyderabad_Siripur-Khagaj Nagar','Hyderabad_Kothakota','Hyderabad_Huzur Nagar','Hyderabad_Kadiri','Hyderabad_Hindupur','Hyderabad_Kakinada','Hyderabad_nalgonda','Hyderabad_Guntur','Ongole VHYD','Tirupathi VHYD','Warangal VHYD','Hyderabad_Kurnool', 'Hyderabad_Ranathalam', 'Hyderabad', 'Hyderabad_Vizianagram', 'VISHAKHAPATNAM', 'Vijayawada', 'Hyderabad_', 'Ananthapur VHYD', 'Hyderabad_Kakinada', 'Rajamundhry VHYD', 'Hyderabad_Tirupathi', 'Hyderabad_Nellore Venkataramapuram', 'Hyderabad_Karimnagar', 'Hyderabad_Kadiri', 'Hyderabad_Nellore', 'Hyderabad_Hindupur', 'Hyderabad_Kakinada', 'Hyderabad_nalgonda', 'Hyderabad_Guntur', 'Guntur VHYD', 'Hyderabad_Badvel', 'Hyderabad_Khammam Grean Markt Road', 'Hyderabad_Nizamabad', 'Hyderabad_Adilabad', 'Hyderabad_Warangal karimabad', 'Hyderabad_Hyderabad', 'Hyderabad_Mancherial', 'Kakinada VHYD', 'Hyderabad_Rajamundry Thadithota', 'Hyderabad_CUDDAPAH', 'Hyderabad_Warangal', 'Cuddapah VHYD', 'Khammam VHYD', 'Srikakulam VHYD', 'Rajamundhry VHYD', 'Nizamabad VHYD', 'Nellore VHYD', 'Vizianagaram VHYD', 'Ananthapur VHYD', 'VISHAKHAPATNAM', 'Vijayawada','Kurnool VHYD','Hyderabad','Mahabub Nagar VHYD','Hyderabad_Vizianagram','Hyderabad_Kurnool','Hyderabad_Vijayawada','Adilabad VHYD','Hyderabad_kalyandurg','Hyderabad_Dharamavaram','Hyderabad_Jubilee Hills','Kurnool VHYD','Hyderabad','Mahabub Nagar VHYD','Hyderabad_Vizianagram','Hyderabad_Kurnool','Hyderabad_Vijayawada','Adilabad VHYD','Hyderabad_kalyandurg','Hyderabad_Dharamavaram','Hyderabad_Jubilee Hills',],
    'Bangalore': ['MANGALORE VBLR','BIDAR VBLR','BAGALKOT VBLR','MYSORE VBLR','BELGAVI VBLR','VIJAYAPURA VBLR','PALAGHAT VBLR','DAVENGERE VBLR','TUMKUR VBLR','TRIVANDRUM VBLR','KANNUR VBLR','HUBLI VBLR','CHITRADURGA VBLR','Bangalore', 'COCHIN VBLR', 'MYSORE VBLR', 'TRIVANDRUM VBLR', 'MANGALORE VBLR', 'KALABURGI VBLR', 'DAVENGERE VBLR', 'SHIMOGA VBLR', 'CALICUT VBLR', 'BELGAVI VBLR', 'HUBLI VBLR', 'TRICHUR VBLR', 'BELLARY VBLR', 'HOSPET VBLR'],
    'Kolkata': ['CHACHAR VKOL','JAJPUR VKOL','JHARKHAND VKOL','BHUBANESHWAR VKOL', 'SILIGURI VKOL', 'RANCHI VKOL', 'Kolkata', 'GUWAHATI VKOL', 'PATNA VKOL'],
    'Chennai': ['KRISHNAGIRI VMAA','KARUR VMAA''DINDIGUL VMAA','TIRUPPUR VMAA','TRICHY HUB VMAA','Chennai', 'SALEM HUB VMAA', 'COIMBATORE VMAA', 'MADURAI HUB VMAA', 'KANCHIPURAM VMAA', 'CHENGALPATTU VMAA', 'PONDICHERRY VMAA', 'ERODE VMAA', 'VELLORE VMAA', 'TIRUPPUR VMAA', 'TIRUNELVELI HUB VMAA'],
    'Delhi NCR': ['Farukh Nagar','Delhi_LAXMI MARKET MUNIRKA','NOIDA','Azamgarh VDEL','Alipur','Mahipalpur','Delhi_SADAR BAZAR','Ghaziabagd VDEL', 'Ambala VDEL', 'Rohtak VDEL','Allahabad VDEL','Delhi'],
    'Rest Delhi':['Delhi_Sindhi Colony Bani Park','Delhi_Chandigarh','Ludhiana VDEL', 'Jammu VDEL'],
    'Nagpur': [ 'Ujjain VNGP','Khandwa VNGP','Attara VNGP','Ambikapur VNGP','Balod VNGP','Bhanpuri VNGP','Bhatapara VNGP','Bhilai VNGP','Bilaspur VNGP','Bilha VNGP','Birgaon VNGP','Bodri VNGP','Champa VNGP','Dhamtari VNGP','Durg VNGP','Jagdalpur VNGP','Janjgir VNGP','Korba VNGP','Mahasamund VNGP','Raigarh VNGP','Rajnandgaon VNGP','Sahadol VNGP','Nagpur', 'Indore VNGP', 'Raipur VNGP', 'Jabalpur VNGP', 'Bhopal VNGP', 'Raigarh VNGP', 'Chhindwara VNGP'],
    'Lucknow': ['Allahabad VDEL','Lucknow', 'Lucknow_Kanpur','Varanasi VDEL'],
    'Ahmedabad': ['BANASKANTHA VAHM','GODHARA VAHM','JUNAGADH VAHM','NADIAD VAHM','PORBANDAR VAHM','SABARKANTHA VAHM','SURENDRANAGAR VAHM','SURAT','Vapi','VADODARA VAHM','GANDHIDHAM-KUTCH VAHM','Ahmedabad_9227207320','Ahmedabad_NEW MANINAGAR','Ahmedabad_Isanpur','Ahmedabad','VADODARA VAHM','GANDHIDHAM-KUTCH VAHM','Ahmedabad_9227207320','Ahmedabad_NEW MANINAGAR','Ahmedabad_Isanpur','Ahmedabad', 'ANKLESHWAR VAHM', 'VADODARA VAHM', 'RAJKOT VAHM', 'GANDHINAGAR VAHM', 'MEHSANA VAHM', 'DAHEJ VAHM', 'JAMNAGAR VAHM', 'BHUJ VAHM'],
    'Pune': ['Goa VBHWD','Pune VADGAON','Pune VADGAON_Aurangabad','Pune WADAKI','Satara VPNQV','Karad VPNQV','Sangali VPNQV','Baramati VPNQV','Daund VPNQV','Kolhapur VPNQV','Ahamad Nagar VPNQV','Kurkumbh VPNQV''Pune VADGAON'],
    'Mumbai': ['Mumbai CHANDIVALI (HUB)','Mumbai CHANDIVALI (HUB)_Govandi','Thane VBHWD','Kalamboli VBHWD','Goregaon BR','Bhiwandi (HUB)','Vasai VBHWD','Vashi VBHWD','Mumbai CHANDIVALI (HUB)_CBD Belapur']
}

# Create a dictionary to hold the count values
all_data = {'City': [],'SEPT': [],'OCT':[],'NOV':[],'DEC':[]}

# Process data for each city and update the dictionary
for city_name, city_branches in cities.items():
    df_city = process_city(city_name, city_branches)
    sep_count = df_city[df_city['Booking Date'].dt.month == 9].shape[0]
    oct_count = df_city[df_city['Booking Date'].dt.month == 10].shape[0]
    nov_count = df_city[df_city['Booking Date'].dt.month == 11].shape[0]
    dec_count = df_city[df_city['Booking Date'].dt.month == 12].shape[0]
    
    
    all_data['City'].append(city_name)
    all_data['SEPT'].append(sep_count)
    all_data['OCT'].append(oct_count)
    all_data['NOV'].append(nov_count)
    all_data['DEC'].append(dec_count)
    

# Create a DataFrame from the dictionary
count_df = pd.DataFrame(all_data)


count_df['Sum'] = count_df[['SEPT', 'OCT', 'NOV','DEC']].sum(axis=1)


# Set display format for float values to remove decimal places
pd.options.display.float_format = '{:.0f}'.format

# Create DataFrames for each city
hyd_filtered = process_city('Hyderabad', cities['Hyderabad'])
bgl_filtered = process_city('Bangalore', cities['Bangalore'])
kol_filtered = process_city('Kolkata', cities['Kolkata'])
maa_filtered = process_city('Chennai', cities['Chennai'])
del_filtered = process_city('Delhi NCR', cities['Delhi NCR'])
res_del_filtered = process_city('Rest Delhi', cities['Rest Delhi'])
nag_filtered = process_city('Nagpur', cities['Nagpur'])
luk_filtered = process_city('Lucknow', cities['Lucknow'])
amd_filtered = process_city('Ahmedabad', cities['Ahmedabad'])
pune_filtered = process_city('Pune', cities['Pune'])
bhi_filtered = process_city('Mumbai', cities['Mumbai'])

# Create a dictionary to hold region-wise data
regions = {
    'Hyderabad': hyd_filtered,
    'Bangalore': bgl_filtered,
    'Kolkata': kol_filtered,
    'Chennai': maa_filtered,
    'Rest Delhi': res_del_filtered,
    'Delhi NCR': del_filtered,
    'Nagpur': nag_filtered,
    'Lucknow': luk_filtered,
    'Ahmedabad': amd_filtered,
    'Pune': pune_filtered,
    'Mumbai': bhi_filtered
}

In [6]:
count_df

,City,SEPT,OCT,NOV,DEC,Sum
0,Hyderabad,0,18,30,10,58
1,Bangalore,0,25,2,0,27
2,Kolkata,0,35,49,21,105
3,Chennai,0,0,0,6,6
4,Delhi NCR,0,0,0,2,2
5,Rest Delhi,0,0,0,0,0
6,Nagpur,0,0,2,0,2
7,Lucknow,0,0,0,11,11
8,Ahmedabad,0,0,0,1,1
9,Pune,0,0,0,1,1


In [7]:
count_df.to_excel(f'Count Of PODs Pending_{current_date}.xlsx')

In [8]:
# Define recipient and CC email addresses, and email template for each city
email_info = {
    'Hyderabad': {
        'subject': "HYDERABAD SOFTCOPY POD's  PENDING UPDATION",
        'to_email': ['dileep.s@boxnfreight.com','priya.z@boxnfreight.com', 'ops.hyderabad@boxnfreight.com', 'ops.vijaywada@boxnfreight.com', 'ops.vizag@boxnfreight.com'],
        'cc_email': ['chand.s@boxnfreight.com'],
        'body': """\
            <p>Dear Dileep Ji and Priya Ma’am,</p>
            <p>Kindly prioritize the closure of the below-mentioned PODs for Hyderabad </p>
            {html_table}


            <p style="color: #3366cc; font-size: 18px; font-weight: bold;">Best Regards,</p>
            <p style="color: #3366cc; font-size: 16px; font-weight: bold;">Sanil Kumar Barik</p>
            <p style="color: #C45911; font-size: 16px; font-weight: bold;">Executive Operations Analyst</p>
            <p style="color: #3366cc; font-size: 16px; font-weight: bold;">+91 9987763071</p>
            <p style="color: #990099; font-size: 16px; font-weight: bold;">opscoordinator@boxnfreight.com</p>
            <p style="color: #1F4E79; font-size: 16px; font-weight: bold;">BOX <span style="color:#C45911">N</span> FREIGHT LOGISTICS SOLUTION PVT LTD</p>
            <p style="color: #0070C0; font-size: 16px; font-weight: bold;">133 &amp; 134, Udyog Bhavan, Sonawala Road,</p>
            <p style="color: #0070C0; font-size: 16px; font-weight: bold;">Goregaon East, Mumbai – 400063</p>
        """
    },
    
    'Bangalore': {
        'subject': "BANGALORE SOFTCOPY POD's  PENDING UPDATION",
        'to_email': ['venkatesh.k@boxnfreight.com', 'shiva.k@boxnfreight.com', 'ops.bengaluru@boxnfreight.com'],
        'cc_email': ['chand.s@boxnfreight.com'],
        'body': """\
            <p>Dear Venkatesh Ji,</p>
            <p>Kindly prioritize the closure of the below-mentioned PODs for Bangalore</p>
            {html_table}


            <p style="color: #3366cc; font-size: 18px; font-weight: bold;">Best Regards,</p>
            <p style="color: #3366cc; font-size: 16px; font-weight: bold;">Sanil Kumar Barik</p>
            <p style="color: #C45911; font-size: 16px; font-weight: bold;">Executive Operations Analyst</p>
            <p style="color: #3366cc; font-size: 16px; font-weight: bold;">+91 9987763071</p>
            <p style="color: #990099; font-size: 16px; font-weight: bold;">opscoordinator@boxnfreight.com</p>
            <p style="color: #1F4E79; font-size: 16px; font-weight: bold;">BOX <span style="color:#C45911">N</span> FREIGHT LOGISTICS SOLUTION PVT LTD</p>
            <p style="color: #0070C0; font-size: 16px; font-weight: bold;">133 &amp; 134, Udyog Bhavan, Sonawala Road,</p>
            <p style="color: #0070C0; font-size: 16px; font-weight: bold;">Goregaon East, Mumbai – 400063</p>
        """
    },
    
    'Kolkata': {
        'subject': "KOLKATA SOFTCOPY POD's  PENDING UPDATION",
        'to_email': ['sandip@boxnfreight.com', 'ops.Kolkata@boxnfreight.com'],
        'cc_email': ['chand.s@boxnfreight.com'],
        'body': """\
            <p>Dear Sandip Ji,</p>
            <p>Kindly prioritize the closure of the below-mentioned PODs for Kolkata </p>
            {html_table}


            <p style="color: #3366cc; font-size: 18px; font-weight: bold;">Best Regards,</p>
            <p style="color: #3366cc; font-size: 16px; font-weight: bold;">Sanil Kumar Barik</p>
            <p style="color: #C45911; font-size: 16px; font-weight: bold;">Executive Operations Analyst</p>
            <p style="color: #3366cc; font-size: 16px; font-weight: bold;">+91 9987763071</p>
            <p style="color: #990099; font-size: 16px; font-weight: bold;">opscoordinator@boxnfreight.com</p>
            <p style="color: #1F4E79; font-size: 16px; font-weight: bold;">BOX <span style="color:#C45911">N</span> FREIGHT LOGISTICS SOLUTION PVT LTD</p>
            <p style="color: #0070C0; font-size: 16px; font-weight: bold;">133 &amp; 134, Udyog Bhavan, Sonawala Road,</p>
            <p style="color: #0070C0; font-size: 16px; font-weight: bold;">Goregaon East, Mumbai – 400063</p>
        """
    },
    
    'Chennai': {
        'subject': "CHENNAI SOFTCOPY POD's  PENDING UPDATION",
        'to_email': ['vimal.k@boxnfreight.com', 'ops.chennai@boxnfreight.com','singaravelu.t@boxnfreight.com'],
        'cc_email': ['chand.s@boxnfreight.com'],
        'body': """\
            <p>Dear Vimal Ji,</p>
            <p>Kindly prioritize the closure of the below-mentioned PODs for Chennai </p>
            {html_table}


            <p style="color: #3366cc; font-size: 18px; font-weight: bold;">Best Regards,</p>
            <p style="color: #3366cc; font-size: 16px; font-weight: bold;">Sanil Kumar Barik</p>
            <p style="color: #C45911; font-size: 16px; font-weight: bold;">Executive Operations Analyst</p>
            <p style="color: #3366cc; font-size: 16px; font-weight: bold;">+91 9987763071</p>
            <p style="color: #990099; font-size: 16px; font-weight: bold;">opscoordinator@boxnfreight.com</p>
            <p style="color: #1F4E79; font-size: 16px; font-weight: bold;">BOX <span style="color:#C45911">N</span> FREIGHT LOGISTICS SOLUTION PVT LTD</p>
            <p style="color: #0070C0; font-size: 16px; font-weight: bold;">133 &amp; 134, Udyog Bhavan, Sonawala Road,</p>
            <p style="color: #0070C0; font-size: 16px; font-weight: bold;">Goregaon East, Mumbai – 400063</p>
        """
    },

    'Rest Delhi': {
        'subject': "Rest Delhi SOFTCOPY POD's  PENDING UPDATION",
        'to_email': ['mohit.t@boxnfreight.com'],
        'cc_email': ['chand.s@boxnfreight.com','amitesh.k@boxnfreight.com'],
        'body': """\
            <p>Dear Mohit Ji,</p>
            <p>Kindly prioritize the closure of the below-mentioned PODs for Rest Delhi </p>
            {html_table}


            <p style="color: #3366cc; font-size: 18px; font-weight: bold;">Best Regards,</p>
            <p style="color: #3366cc; font-size: 16px; font-weight: bold;">Sanil Kumar Barik</p>
            <p style="color: #C45911; font-size: 16px; font-weight: bold;">Executive Operations Analyst</p>
            <p style="color: #3366cc; font-size: 16px; font-weight: bold;">+91 9987763071</p>
            <p style="color: #990099; font-size: 16px; font-weight: bold;">opscoordinator@boxnfreight.com</p>
            <p style="color: #1F4E79; font-size: 16px; font-weight: bold;">BOX <span style="color:#C45911">N</span> FREIGHT LOGISTICS SOLUTION PVT LTD</p>
            <p style="color: #0070C0; font-size: 16px; font-weight: bold;">133 &amp; 134, Udyog Bhavan, Sonawala Road,</p>
            <p style="color: #0070C0; font-size: 16px; font-weight: bold;">Goregaon East, Mumbai – 400063</p>
        """
    },

    'Delhi NCR': {
        'subject': "DELHI SOFTCOPY POD's  PENDING UPDATION",
        'to_email': ['bhupendra.s@boxnfreight.com'],
        'cc_email': ['chand.s@boxnfreight.com','amitesh.k@boxnfreight.com'],
        'body': """\
            <p>Dear Bhupendra Ji,</p>
            <p>Kindly prioritize the closure of the below-mentioned PODs for Delhi NCR</p>
            {html_table}


            <p style="color: #3366cc; font-size: 18px; font-weight: bold;">Best Regards,</p>
            <p style="color: #3366cc; font-size: 16px; font-weight: bold;">Sanil Kumar Barik</p>
            <p style="color: #C45911; font-size: 16px; font-weight: bold;">Executive Operations Analyst</p>
            <p style="color: #3366cc; font-size: 16px; font-weight: bold;">+91 9987763071</p>
            <p style="color: #990099; font-size: 16px; font-weight: bold;">opscoordinator@boxnfreight.com</p>
            <p style="color: #1F4E79; font-size: 16px; font-weight: bold;">BOX <span style="color:#C45911">N</span> FREIGHT LOGISTICS SOLUTION PVT LTD</p>
            <p style="color: #0070C0; font-size: 16px; font-weight: bold;">133 &amp; 134, Udyog Bhavan, Sonawala Road,</p>
            <p style="color: #0070C0; font-size: 16px; font-weight: bold;">Goregaon East, Mumbai – 400063</p>
        """
    },

    'Nagpur': {
        'subject': "NAGPUR SOFTCOPY POD's  PENDING UPDATION",
        'to_email': ['ops.nagpur@boxnfreight.com','abhishek.k@boxnfreight.com', 'nilesh.g@boxnfreight.com'],
        'cc_email': ['chand.s@boxnfreight.com','hanif.s@boxnfreight.com'],
        'body': """\
            <p>Dear Nilesh Ji,</p>
            <p>Kindly prioritize the closure of the below-mentioned PODs for Nagpur </p>
            {html_table}


            <p style="color: #3366cc; font-size: 18px; font-weight: bold;">Best Regards,</p>
            <p style="color: #3366cc; font-size: 16px; font-weight: bold;">Sanil Kumar Barik</p>
            <p style="color: #C45911; font-size: 16px; font-weight: bold;">Executive Operations Analyst</p>
            <p style="color: #3366cc; font-size: 16px; font-weight: bold;">+91 9987763071</p>
            <p style="color: #990099; font-size: 16px; font-weight: bold;">opscoordinator@boxnfreight.com</p>
            <p style="color: #1F4E79; font-size: 16px; font-weight: bold;">BOX <span style="color:#C45911">N</span> FREIGHT LOGISTICS SOLUTION PVT LTD</p>
            <p style="color: #0070C0; font-size: 16px; font-weight: bold;">133 &amp; 134, Udyog Bhavan, Sonawala Road,</p>
            <p style="color: #0070C0; font-size: 16px; font-weight: bold;">Goregaon East, Mumbai – 400063</p>
        """
    },

    'Lucknow': {
        'subject': "LUCKNOW SOFTCOPY POD's  PENDING UPDATION",
        'to_email': ['ops.lucknow@boxnfreight.com', 'manoj.g@boxnfreight.com'],
        'cc_email': ['chand.s@boxnfreight.com'],
        'body': """\
            <p>Dear Manoj Ji,</p>
            <p>Kindly prioritize the closure of the below-mentioned PODs for Lucknow </p>
            {html_table}


            <p style="color: #3366cc; font-size: 18px; font-weight: bold;">Best Regards,</p>
            <p style="color: #3366cc; font-size: 16px; font-weight: bold;">Sanil Kumar Barik</p>
            <p style="color: #C45911; font-size: 16px; font-weight: bold;">Executive Operations Analyst</p>
            <p style="color: #3366cc; font-size: 16px; font-weight: bold;">+91 9987763071</p>
            <p style="color: #990099; font-size: 16px; font-weight: bold;">opscoordinator@boxnfreight.com</p>
            <p style="color: #1F4E79; font-size: 16px; font-weight: bold;">BOX <span style="color:#C45911">N</span> FREIGHT LOGISTICS SOLUTION PVT LTD</p>
            <p style="color: #0070C0; font-size: 16px; font-weight: bold;">133 &amp; 134, Udyog Bhavan, Sonawala Road,</p>
            <p style="color: #0070C0; font-size: 16px; font-weight: bold;">Goregaon East, Mumbai – 400063</p>
        """
    },

    'Ahmedabad': {
        'subject': "AHMEDABAD SOFTCOPY POD's  PENDING UPDATION",
        'to_email': ['harish.s@boxnfreight.com', 'anil.p@boxnfreight.com'],
        'cc_email': ['chand.s@boxnfreight.com'],
        'body': """\
            <p>Dear Harish Ji/ Anil Ji,</p>
            <p>Kindly prioritize the closure of the below-mentioned PODs for Ahmedabad </p>
            {html_table}


            <p style="color: #3366cc; font-size: 18px; font-weight: bold;">Best Regards,</p>
            <p style="color: #3366cc; font-size: 16px; font-weight: bold;">Sanil Kumar Barik</p>
            <p style="color: #C45911; font-size: 16px; font-weight: bold;">Executive Operations Analyst</p>
            <p style="color: #3366cc; font-size: 16px; font-weight: bold;">+91 9987763071</p>
            <p style="color: #990099; font-size: 16px; font-weight: bold;">opscoordinator@boxnfreight.com</p>
            <p style="color: #1F4E79; font-size: 16px; font-weight: bold;">BOX <span style="color:#C45911">N</span> FREIGHT LOGISTICS SOLUTION PVT LTD</p>
            <p style="color: #0070C0; font-size: 16px; font-weight: bold;">133 &amp; 134, Udyog Bhavan, Sonawala Road,</p>
            <p style="color: #0070C0; font-size: 16px; font-weight: bold;">Goregaon East, Mumbai – 400063</p>
        """
    },

    'Pune': {
        'subject': "PUNE SOFTCOPY POD's PENDING UPDATION",
        'to_email': ['vaibhav.s@boxnfreight.com', 'ops.punewadgaon@boxnfreight.com'],
        'cc_email': ['chand.s@boxnfreight.com','hanif.s@boxnfreight.com'],
        'body': """\
            <p>Dear Team,</p>
            <p>Kindly prioritize the closure of the below-mentioned PODs for Pune </p>
            {html_table}


            <p style="color: #3366cc; font-size: 18px; font-weight: bold;">Best Regards,</p>
            <p style="color: #3366cc; font-size: 16px; font-weight: bold;">Sanil Kumar Barik</p>
            <p style="color: #C45911; font-size: 16px; font-weight: bold;">Executive Operations Analyst</p>
            <p style="color: #3366cc; font-size: 16px; font-weight: bold;">+91 9987763071</p>
            <p style="color: #990099; font-size: 16px; font-weight: bold;">opscoordinator@boxnfreight.com</p>
            <p style="color: #1F4E79; font-size: 16px; font-weight: bold;">BOX <span style="color:#C45911">N</span> FREIGHT LOGISTICS SOLUTION PVT LTD</p>
            <p style="color: #0070C0; font-size: 16px; font-weight: bold;">133 &amp; 134, Udyog Bhavan, Sonawala Road,</p>
            <p style="color: #0070C0; font-size: 16px; font-weight: bold;">Goregaon East, Mumbai – 400063</p>
        """
    },

      'Mumbai': {
        'subject': "MUMBAI SOFTCOPY POD's PENDING UPDATION",
        'to_email': ['vishal.k@boxnfreight.com', 'ops.bhiwandi@boxnfreight.com','tushar.s@boxnfreight.com'],
        'cc_email': ['chand.s@boxnfreight.com','hanif.s@boxnfreight.com'],
        'body': """\
            <p>Dear Vishal Ji/Tushar Ji,</p>
            <p>Kindly prioritize the closure of the below-mentioned PODs for Mumbai </p>
            {html_table}

            
            <p style="color: #3366cc; font-size: 18px; font-weight: bold;">Best Regards,</p>
            <p style="color: #3366cc; font-size: 16px; font-weight: bold;">Sanil Kumar Barik</p>
            <p style="color: #C45911; font-size: 16px; font-weight: bold;">Executive Operations Analyst</p>
            <p style="color: #3366cc; font-size: 16px; font-weight: bold;">+91 9987763071</p>
            <p style="color: #990099; font-size: 16px; font-weight: bold;">opscoordinator@boxnfreight.com</p>
            <p style="color: #1F4E79; font-size: 16px; font-weight: bold;">BOX <span style="color:#C45911">N</span> FREIGHT LOGISTICS SOLUTION PVT LTD</p>
            <p style="color: #0070C0; font-size: 16px; font-weight: bold;">133 &amp; 134, Udyog Bhavan, Sonawala Road,</p>
            <p style="color: #0070C0; font-size: 16px; font-weight: bold;">Goregaon East, Mumbai – 400063</p>
        """
    }
}

# Function to send email using Outlook
def send_email(to_email, cc_email, subject, message):
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = ';'.join(to_email)
    mail.CC = ';'.join(cc_email)
    mail.Subject = subject
    mail.HTMLBody = message
    mail.Send()

# Send region-wise emails
for region_name, region_data in regions.items():
    if not region_data.empty:  # Check if the DataFrame is not empty
        email_details = email_info.get(region_name)
        if email_details:
            # Convert the DataFrame to an HTML table with styling
            html_table = region_data.to_html(classes='table table-bordered table-striped', escape=False, index=False)

            # Apply styling to the HTML table (column headers in light blue)
            styled_html_table = html_table.replace('<thead>', '<thead style="background-color: lightblue;">')
            
            # Get email details
            subject = email_details['subject']
            to_email = email_details['to_email']
            cc_email = email_details['cc_email']
            body = email_details['body'].format(current_date=current_date, html_table=styled_html_table)

            # Send the email
            try:
                send_email(to_email, cc_email, subject, body)
                print(f"Email sent to {', '.join(to_email)} and {', '.join(cc_email)}")
            except Exception as e:
                print(f"Error sending email to {', '.join(to_email)} and {', '.join(cc_email)}: {e}")
    else:
        print(f"No data for {region_name}. Email not sent.")

Email sent to chand.s@boxnfreight.com and chand.s@boxnfreight.com
No data for Rest Delhi. Email not sent.
No data for Mumbai. Email not sent.
